In [ ]:
import inspect
import multiprocessing as mp
import numpy as np
import os
import threading

%load_ext autoreload
%autoreload 2

In [ ]:
from msbd.scraping import ottieni_contenuto_url

print(inspect.getsource(ottieni_contenuto_url))

In [ ]:
contenuto = ottieni_contenuto_url("https://www.didattica.unipd.it/off/2016/LT/SC/SC2094/000ZZ/SCP4063754/N0")

In [ ]:
URLS = [
    "https://en.wikipedia.org/wiki/Multithreading_(computer_architecture)",
    "https://en.wikipedia.org/wiki/Multiprocessing",
    "https://docs.python.org/3.6/library/threading.html",
    "https://docs.python.org/3.6/library/multiprocessing.html",
]

### Esercizio

# `Thread`

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
def salva_contenuto_urls_multiprocess(urls, file):
    coda = mp.Queue()
    
    processi = [mp.Process(target=ottieni_contenuto_url, args=(url, coda)) 
        for url in urls]
    
    for p in processi:
        p.start()

    contenuto = [coda.get() for p in processi]
    
    for p in processi:
        p.join()

    return contenuto

In [ ]:
prova[0]

In [ ]:
prova = salva_contenuto_urls_multiprocess(urls, "aaa")

In [ ]:
def contenuto_url(url, coda=None):
    id_processo = os.getpid()
    nome_processo = mp.current_process().name
    nome_thread = threading.current_thread().name
    
    print("[INIZIO]\nID processo: {}\nNome processo: {}\nNome thread: {}\n".format(
        id_processo, nome_processo, nome_thread))

    try:
        risposta = requests.get(url)
        testo = BeautifulSoup(risposta.content, "lxml").get_text()
    except:
        testo = ""
        
    print("[FINE]\nID processo: {}\nNome processo: {}\nNome thread: {}\n".format(
        id_processo, nome_processo, nome_thread))
    
    if coda is not None:
        coda.put(testo)
    else:
        return testo

In [ ]:
urls = [
    "https://it.wikipedia.org/wiki/Multithreading",
    "https://docs.python.org/3.6/library/threading.html",
    "https://realpython.com/intro-to-python-threading/",
    "https://stackoverflow.com/questions/3044580/multiprocessing-vs-threading-python"
]

In [ ]:
from msbd.scraping.scraping import ottieni_contenuto_url

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
n_cpu = mp.cpu_count()

print("# CPU: {}".format(n_cpu))

In [ ]:
risposta = requests.get(urls[0])

In [ ]:
testo = BeautifulSoup(risposta.content, "lxml").get_text()

In [ ]:
        risposta = requests.get(urls[0])
        testo = BeautifulSoup(risposta.content, "lxml").get_text()

In [ ]:
        risposta = requests.get(urls[0])
        testo = BeautifulSoup(risposta.content, "lxml").get_text()

In [ ]:
import pickle

In [ ]:
with open("datasets/urls/sequenziale.pkl", "rb") as f:
    contenuti = pickle.load(f)

In [ ]:
OUTPUT_DIR = "output/14"

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
!python msbd/scraping/scraping.py --urls datasets/urls/urls.txt --tipo sequenziale --output=$OUTPUT_DIR/sequenziale.pkl

In [ ]:
!python msbd/scraping/scraping.py --urls datasets/urls/urls.txt --tipo multiprocess --output=$OUTPUT_DIR/multiprocess.pkl

In [ ]:
import time

In [ ]:
with open("datasets/url/lista_url.txt") as f:
    lista_url = f.read().split("\n")

In [ ]:
lista_url

In [ ]:
inizio = time.time()

contenuti = [contenuto_url(url) for url in urls]

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

In [ ]:
def process(target, urls):
    coda = mp.Queue()
    
    processi = [mp.Process(target=target, args=(url, coda)) for url in urls]
    
    for p in processi:
        p.start()
    
    for p in processi:
        p.join()

    return [coda.get() for p in processi] 

In [ ]:
contenuti

In [ ]:
inizio = time.time()

__name__ == "main"

contenuti = process(contenuto_url, urls)

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

In [ ]:
inizio = time.time()

coda_fifo = mp.Queue()
    
processi = [threading.Thread(target=ottieni_testo_da_url, args=(url, coda_fifo)) 
            for url in urls]
    
for p in processi:
    p.start()
    
for p in processi:
    p.join()

testi = [coda_fifo.get() for p in processi] 

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

In [ ]:
inizio = time.time()

contenuti = process()

fine = time.time()
print("Durata: {:.2f}s".format(fine - inizio))

In [ ]:
def e_primo(n):
    
    return (n >= 2) & all(n % i for i in range(2, n))


def conta_numeri_primi(numeri, dormi={}, verboso=False):

    if verboso:
        pid = os.getpid()
        pname = mp.current_process().name
        tname = threading.current_thread().name
        print("PID: {}, Processo: {}, Thread: {}".format(pid, pname, tname))
    
    c = 0
    
    for n in numeri:

        if n in dormi:
            time.sleep(dormi[n])

        if (n >= 2) & all(n % i for i in range(2, n)):
            c += 1

    return c

In [ ]:
def target(numeri, output, dormi, verboso):
    output.put(conta_numeri_primi(numeri, dormi, verboso))

def conta_numeri_primi_parallelo(numeri, classe, dormi={}, verboso=False, 
                                 indices_or_sections=4):
    output = mp.Queue()
    
    processi = [classe(target=target, args=(n, output, dormi, verboso)) 
                for n in np.array_split(numeri, indices_or_sections)]
    
    for p in processi:
        p.start()
    
    for p in processi:
        p.join()

    return sum([output.get() for p in processi])

In [ ]:
n = conta_numeri_primi(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))
print()

In [ ]:
n = conta_numeri_primi_parallelo(numeri, mp.Process, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
n = conta_numeri_primi_parallelo(numeri, threading.Thread, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri, dormi={0: 0.25})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread, dormi={0: 0.25})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process, dormi={0: 0.25})

In [ ]:
def conta_numeri_primi_pool_apply(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum(risultati)

def conta_numeri_primi_pool_apply_async(numeri, dormi={}, verboso=False, 
                            indices_or_sections=4, processes=mp.cpu_count()):
    pool = mp.Pool(processes=processes)
    
    risultati = [pool.apply_async(conta_numeri_primi, args=(n, dormi, verboso)) 
                 for n in np.array_split(numeri, indices_or_sections)] 
    
    return sum([p.get() for p in risultati])

In [ ]:
n = conta_numeri_primi_pool_apply(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
n = conta_numeri_primi_pool_apply_async(numeri, verboso=True)

print("\n# numeri primi: {}".format(n))

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi(numeri)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply_async(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, threading.Thread, dormi={0: 1})

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_parallelo(numeri, mp.Process, dormi={0: 1})

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool_apply(numeri, indices_or_sections=100)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 2 -n 1 conta_numeri_primi_pool_apply_async(numeri, dormi={0: 1}, indices_or_sections=16)

In [ ]:
%timeit -r 10 -n 1 conta_numeri_primi_pool(numeri)

In [ ]:
pool = mp.Pool(processes=4)

risultati = [pool.apply_async(conta_numeri_primi, args=(n, {}, False)) 
             for n in np.array_split(numeri, 100)] 
sum([p.get() for p in risultati])

In [ ]:
pool = mp.Pool(processes=4)
results = [pool.apply_async(cube, args=(x,)) for x in range(1,7)]
output = [p.get() for p in results]
print(output)